## Problem 1: Geocode shopping centers (5 points)

The overall aim of problems 1-3 is to find out **how many people live within a walking distance (1.5 km) from certain shopping centers in Helsinki**.

In problem 1 aim is to find out the addresses of shopping centers and to retreive numercal coordinates for these addresses. As an output, we will have a Shapefile called `shopping_centers.shp` which contains the geocoded result.

**Preparation:** Find out the addresses for following shopping centers from the internet, and write the addresses into a text file called `shopping_centers.txt`:

 - Itis
 - Forum
 - Iso-omena
 - Sello
 - Jumbo
 - REDI
 - Tripla 
 
 *Hint for "Tripla": This shopping center opened in October 10 2019, and if you are doing this exercise soon after that, the official address might not yet be in online databases. 
 Check for an address nearby the Pasila railway station on OpenStreetMap.org and use that as input.*

`shopping_centers.txt` should have semicolon (`;`) as a separator, and the file should include the following columns:

- ``id`` (integer) containing an unique identifier for each shopping center
- ``name`` (string) of each shopping center
- ``addr`` (string) the address 


See and example of how to format the text file [in the lesson 3 materials](https://automating-gis-processes.github.io/site/master/notebooks/L3/geocoding_in_geopandas.html). Save (and upload) the text file into your exercise repository.

- Read `shopping_centers.txt` that you just created into a pandas DataFrame called ``data``:

In [1]:
# Import modules
import pandas as pd
import numpy as np
import geopandas as gpd

# Read the data (replace "None" with your own code)
data = pd.read_csv('shopping_centers.txt', sep=';')

In [2]:
#NON-EDITABLE TEST CELL
# Check your input data
print(data)

   id       name                                            addr
0   0       Itis            Itäkatu 1-7, 00930 Helsinki, Finland
1   1      Forum  Mannerheimintie 14-20, 00100 Helsinki, Finland
2   2  Iso-Omena           Piispansilta 11, 02230 Espoo, Finland
3   3      Sello       Leppävaarankatu 3-9, 02600 Espoo, Finland
4   4      Jumbo               Vantaanportinkatu 3, 01510 Vantaa
5   5       REDI            Hermannin rantatie 5, 00580 Helsinki
6   6     Tripla                 Pasilansilta 11, 00520 Helsinki


- Geocode the addresses using the Nominatim geocoding service. Store the output in a variable called `geo`:

In [3]:
# Geocode the addresses using Nominatim
from geopandas.tools import geocode

geo = geocode(data['addr'], provider='nominatim', user_agent='autogis_HA', timeout=5)


In [4]:
#NON-EDITABLE TEST CELL
# Check the geocoded output
print(geo)

                    geometry   
0  POINT (25.08294 60.21170)  \
1  POINT (24.93828 60.16878)   
2  POINT (24.73834 60.16129)   
3  POINT (24.80928 60.21815)   
4  POINT (24.96282 60.29245)   
5  POINT (24.97963 60.18638)   
6  POINT (24.93089 60.19764)   

                                             address  
0  Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...  
1  Salaattiasema, 14-20, Mannerheimintie, Keskust...  
2  Iso Omena, 11, Piispansilta, Matinkylä, Suur-M...  
3  William K, 3-9, Leppävaarankatu, Säteri, Etelä...  
4  Stockmann, 3, Vantaanportinkatu, Vantaanportti...  
5  Oluthuone Haavi, 5, Hermannin rantatie, Verkko...  
6  Pasilansilta, Keski-Pasila, Pasila, Keskinen s...  


In [5]:
#NON-EDITABLE TEST CELL
# Check the data type (should be a GeoDataFrame!)
print(type(geo))

<class 'geopandas.geodataframe.GeoDataFrame'>


Check that the coordinate reference system of the geocoded result is correctly defined, and **reproject the layer into ETRS GK-25** (EPSG:3879):

In [6]:
geo.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [7]:
geo = geo.to_crs(epsg=3879)
geo.crs

<Projected CRS: EPSG:3879>
Name: ETRS89 / GK25FIN
Axis Info [cartesian]:
- N[north]: Northing (metre)
- E[east]: Easting (metre)
Area of Use:
- name: Finland - nominally onshore between 24°30'E and 25°30'E but may be used in adjacent areas if a municipality chooses to use one zone over its whole extent.
- bounds: (24.5, 59.94, 25.5, 68.9)
Coordinate Operation:
- name: Finland Gauss-Kruger zone 25
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989 ensemble
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [8]:
#NON-EDITABLE TEST CELL
# Check layer crs
print(geo.crs)

EPSG:3879


- Make a table join between the geocoded addresses (``geo``) and the original addresses (``data``) in order to link the numerical coordinates and  the `id` and `name` of each shopping center. 
- Store the output in a variable called ``geodata`` 


In [9]:
# Join the tables
geodata = geo.join(data)

In [10]:
#NON-EDITABLE TEST CELL
# Check the join output
print(geodata.head())

                           geometry   
0  POINT (25504598.602 6677662.109)  \
1  POINT (25496573.542 6672878.360)   
2  POINT (25485470.435 6672070.967)   
3  POINT (25489428.008 6678393.041)   
4  POINT (25497943.932 6686656.982)   

                                             address  id       name   
0  Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...   0       Itis  \
1  Salaattiasema, 14-20, Mannerheimintie, Keskust...   1      Forum   
2  Iso Omena, 11, Piispansilta, Matinkylä, Suur-M...   2  Iso-Omena   
3  William K, 3-9, Leppävaarankatu, Säteri, Etelä...   3      Sello   
4  Stockmann, 3, Vantaanportinkatu, Vantaanportti...   4      Jumbo   

                                             addr  
0            Itäkatu 1-7, 00930 Helsinki, Finland  
1  Mannerheimintie 14-20, 00100 Helsinki, Finland  
2           Piispansilta 11, 02230 Espoo, Finland  
3       Leppävaarankatu 3-9, 02600 Espoo, Finland  
4               Vantaanportinkatu 3, 01510 Vantaa  


- Save the output as a Shapefile called `shopping_centers.shp` 

In [11]:
'''# Define output filepath
out_fp = 'shopping centers.shp'

# Save file
geodata.to_file(out_fp)'''

"# Define output filepath\nout_fp = 'shopping centers.shp'\n\n# Save file\ngeodata.to_file(out_fp)"

In [12]:
#NON-EDITABLE TEST CELL
# Print info about output file
#print("Geocoded output is stored in this file:", out_fp)

## Problem 2: Create buffers around shopping centers (5 points)

Let's continue with our case study and calculate a 1.5 km buffer around the geocoded points. 


- Start by creating a new column called `buffer` to ``geodata`` GeoDataFrame:

In [13]:
#create new column called buffer in geodata
geodata['buffer'] = None

- Calculate a 1.5 km buffer for each geocoded point. Store the buffer geometry in the new `buffer` column.

Here, you can use the [GeoDataFrame buffer() method](http://geopandas.org/geometric_manipulations.html#GeoSeries.buffer), which uses Shapely's [buffer](http://toblerity.org/shapely/manual.html#object.buffer) in the bacground. You only need to use the `distance` -parameter, don't worry about the other parameters.

In [14]:
geodata['buffer'] = geodata.buffer(1500, resolution=16)

In [15]:
#NON-EDITABLE TEST CELL
print(geodata.head())

                           geometry   
0  POINT (25504598.602 6677662.109)  \
1  POINT (25496573.542 6672878.360)   
2  POINT (25485470.435 6672070.967)   
3  POINT (25489428.008 6678393.041)   
4  POINT (25497943.932 6686656.982)   

                                             address  id       name   
0  Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...   0       Itis  \
1  Salaattiasema, 14-20, Mannerheimintie, Keskust...   1      Forum   
2  Iso Omena, 11, Piispansilta, Matinkylä, Suur-M...   2  Iso-Omena   
3  William K, 3-9, Leppävaarankatu, Säteri, Etelä...   3      Sello   
4  Stockmann, 3, Vantaanportinkatu, Vantaanportti...   4      Jumbo   

                                             addr   
0            Itäkatu 1-7, 00930 Helsinki, Finland  \
1  Mannerheimintie 14-20, 00100 Helsinki, Finland   
2           Piispansilta 11, 02230 Espoo, Finland   
3       Leppävaarankatu 3-9, 02600 Espoo, Finland   
4               Vantaanportinkatu 3, 01510 Vantaa   

                   

In [16]:
#NON-EDITABLE TEST CELL
# Check the data type of the first value in the buffer-column
print(type(geodata.at[0,'buffer']))

<class 'shapely.geometry.polygon.Polygon'>


In [17]:
#NON-EDITABLE TEST CELL
# Check the areas of your buffers in km^2
print(round(gpd.GeoSeries(geodata["buffer"]).area / 1000000))

0    7.0
1    7.0
2    7.0
3    7.0
4    7.0
5    7.0
6    7.0
dtype: float64


- Replace the values in `geometry` column with the values of `buffer` column:

In [18]:
geodata['geometry'] = geodata['buffer']

In [19]:
#NON-EDITABLE TEST CELL
print(geodata.head())

                                            geometry   
0  POLYGON ((25506098.602 6677662.109, 25506091.3...  \
1  POLYGON ((25498073.542 6672878.360, 25498066.3...   
2  POLYGON ((25486970.435 6672070.967, 25486963.2...   
3  POLYGON ((25490928.008 6678393.041, 25490920.7...   
4  POLYGON ((25499443.932 6686656.982, 25499436.7...   

                                             address  id       name   
0  Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...   0       Itis  \
1  Salaattiasema, 14-20, Mannerheimintie, Keskust...   1      Forum   
2  Iso Omena, 11, Piispansilta, Matinkylä, Suur-M...   2  Iso-Omena   
3  William K, 3-9, Leppävaarankatu, Säteri, Etelä...   3      Sello   
4  Stockmann, 3, Vantaanportinkatu, Vantaanportti...   4      Jumbo   

                                             addr   
0            Itäkatu 1-7, 00930 Helsinki, Finland  \
1  Mannerheimintie 14-20, 00100 Helsinki, Finland   
2           Piispansilta 11, 02230 Espoo, Finland   
3       Leppävaarankatu 

Optional: at this point, you can drop out unnecessary columns from the geodataframe. In the next problem, 
we will only need these columns: `'id', 'name', 'geometry'`

In [42]:
geodata = geodata[['id', 'name', 'geometry']]
geodata.head(10)

,id,name,geometry
0,0,Itis,"POLYGON ((25506098.602 6677662.109, 25506091.3..."
1,1,Forum,"POLYGON ((25498073.542 6672878.360, 25498066.3..."
2,2,Iso-Omena,"POLYGON ((25486970.435 6672070.967, 25486963.2..."
3,3,Sello,"POLYGON ((25490928.008 6678393.041, 25490920.7..."
4,4,Jumbo,"POLYGON ((25499443.932 6686656.982, 25499436.7..."
5,5,REDI,"POLYGON ((25500369.737 6674838.689, 25500362.5..."
6,6,Tripla,"POLYGON ((25497666.681 6676094.626, 25497659.4..."


## Problem 3: How many people live near shopping centers? (5 points)

Last step in our analysis is to make a spatial join between our buffer layer and population data in order to find out **how many people live near each shopping center**. We will use the same data as we did during [lesson 3](https://automating-gis-processes.github.io/site/notebooks/L3/spatial-join.html#Spatial-join); **a Population Grid** that is available via the HSY wfs. 

Alternatively, you can also download the data from the [Helsinki Region Infoshare (HRI)](https://www.hsy.fi/fi/asiantuntijalle/avoindata/Sivut/AvoinData.aspx?dataID=7) as a shapefile (using wget).

The coordinate reference system of the population grid is **ETRS GK-25 (EPSG:3879)**.


**Steps:**

- Read the population grid into a geodataframe

- Select only the useful columns from the population grid: ``'asukkaita'`` (=population count per grid square) and ``'geometry'`` 

- Make a spatial join between your buffered point layer and population grid layer. Join the information now from buffer layer **into the population grid layer**

- Group the joined layer by shopping center index

- Calculate the sum of population living within 1.5 km for each shopping center.

**Finally:**

- Print out the population living within 1.5 km from each shopping center:

     - Itis
     - Forum
     - Iso-omena
     - Sello
     - Jumbo
     - REDI
     - Tripla
     
**Final print out should contain both the shopping center name and population count**, for example: `26532 people live within 1.5 km from Iso-Omena`.

*Feel free to divide your solution into several codeblocks! Remember to comment your code  :)*

In [21]:
# Read population grid data for 2018 into a variable `pop`. 
# Remember to check the crs info! 
from pyproj import CRS
import requests
import geojson

# Specify the url for web feature service
url = 'https://kartta.hsy.fi/geoserver/wfs'

# Specify parameters (read data in json format). 
# Available feature types in this particular data source: http://geo.stat.fi/geoserver/vaestoruutu/wfs?service=wfs&version=2.0.0&request=describeFeatureType
params = dict(service='WFS', 
              version='2.0.0', 
              request='GetFeature', 
              typeName='asuminen_ja_maankaytto:Vaestotietoruudukko_2018', 
              outputFormat='json')

# Fetch data from WFS using requests
r = requests.get(url, params=params)

# Create GeoDataFrame from geojson
pop = gpd.GeoDataFrame.from_features(geojson.loads(r.content))

In [22]:
#check contents
pop.head()

,geometry,index,asukkaita,asvaljyys,ika0_9,ika10_19,ika20_29,ika30_39,ika40_49,ika50_59,ika60_69,ika70_79,ika_yli80
0,"POLYGON ((25472499.995 6689749.005, 25472499.9...",688,9,28.0,99,99,99,99,99,99,99,99,99
1,"POLYGON ((25472499.995 6685998.998, 25472499.9...",703,5,51.0,99,99,99,99,99,99,99,99,99
2,"POLYGON ((25472499.995 6684249.004, 25472499.9...",710,8,44.0,99,99,99,99,99,99,99,99,99
3,"POLYGON ((25472499.995 6683999.005, 25472499.9...",711,5,90.0,99,99,99,99,99,99,99,99,99
4,"POLYGON ((25472499.995 6682998.998, 25472499.9...",715,11,41.0,99,99,99,99,99,99,99,99,99


In [28]:
#select columns needed
pop = pop[['geometry', 'asukkaita']]
pop.rename(columns={'asukkaita' : 'population'}, inplace=True)
pop.head()

,geometry,population
0,"POLYGON ((25472499.995 6689749.005, 25472499.9...",9
1,"POLYGON ((25472499.995 6685998.998, 25472499.9...",5
2,"POLYGON ((25472499.995 6684249.004, 25472499.9...",8
3,"POLYGON ((25472499.995 6683999.005, 25472499.9...",5
4,"POLYGON ((25472499.995 6682998.998, 25472499.9...",11


In [26]:
#check crs
pop.crs

In [29]:
#define crs
pop.crs = CRS.from_epsg(3879)
pop.crs

<Projected CRS: EPSG:3879>
Name: ETRS89 / GK25FIN
Axis Info [cartesian]:
- N[north]: Northing (metre)
- E[east]: Easting (metre)
Area of Use:
- name: Finland - nominally onshore between 24°30'E and 25°30'E but may be used in adjacent areas if a municipality chooses to use one zone over its whole extent.
- bounds: (24.5, 59.94, 25.5, 68.9)
Coordinate Operation:
- name: Finland Gauss-Kruger zone 25
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989 ensemble
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [30]:
#NON-EDITABLE TEST CELL
# Check your input data
print("Number of rows:", len(pop))
print(pop.head(3))

Number of rows: 5832
                                            geometry  population
0  POLYGON ((25472499.995 6689749.005, 25472499.9...           9
1  POLYGON ((25472499.995 6685998.998, 25472499.9...           5
2  POLYGON ((25472499.995 6684249.004, 25472499.9...           8


In [82]:
# Create a spatial join between grid layer and buffer layer. 
join = gpd.sjoin(pop, geodata,  how='right', predicate='within')
join.head()

,index_left,population,id,name,geometry
0,5230,59,0,Itis,"POLYGON ((25506098.602 6677662.109, 25506091.3..."
0,5169,147,0,Itis,"POLYGON ((25506098.602 6677662.109, 25506091.3..."
0,5229,31,0,Itis,"POLYGON ((25506098.602 6677662.109, 25506091.3..."
0,5168,186,0,Itis,"POLYGON ((25506098.602 6677662.109, 25506091.3..."
0,5228,82,0,Itis,"POLYGON ((25506098.602 6677662.109, 25506091.3..."


In [83]:
#group by shopping center index
join_group = join.groupby('name')

In [84]:
# Report how many people live within 1.5 km distance from each index
total_pop = join_group['population'].sum()
total_pop

name
Forum        56891
Iso-Omena    26390
Itis         21008
Jumbo        10569
REDI         24260
Sello        23570
Tripla       23678
Name: population, dtype: int64

**Reflections:**
    
- How challenging did you find problems 1-3 (on scale to 1-5), and why?
- What was easy?
- What was difficult?

YOUR ANSWER HERE

Well done! Now you can continue to [problem 4](Exercise-3-Problem-4.ipynb)